In [1]:
import pandas as pd

In [2]:
raw = pd.read_csv('../data/raw/student/student-mat.csv')
raw.head()

,sex,age,address,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,activities,higher,health,absences,M1,M2,M3
0,F,18,U,4,4,at_home,teacher,2,2,0,no,yes,3,6,25,30,30
1,F,17,U,1,1,at_home,other,1,2,0,no,yes,3,4,25,25,30
2,F,15,U,1,1,at_home,other,1,2,3,no,yes,3,10,35,40,50
3,F,15,U,4,2,health,services,1,3,0,yes,yes,5,2,75,70,75
4,F,16,U,3,3,other,other,1,2,0,no,yes,5,4,30,50,50


In [3]:
raw.head()

,sex,age,address,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,activities,higher,health,absences,M1,M2,M3
0,F,18,U,4,4,at_home,teacher,2,2,0,no,yes,3,6,25,30,30
1,F,17,U,1,1,at_home,other,1,2,0,no,yes,3,4,25,25,30
2,F,15,U,1,1,at_home,other,1,2,3,no,yes,3,10,35,40,50
3,F,15,U,4,2,health,services,1,3,0,yes,yes,5,2,75,70,75
4,F,16,U,3,3,other,other,1,2,0,no,yes,5,4,30,50,50


In [4]:
num_cat = ['M1', 'M2', 'M3']
one_hot_cat = ['sex', 'address', 'Mjob', 'Fjob', 'activities', 'higher']
one_hot_cat_cols = [0, 2, 5, 6, 10, 11]

transformed_data = pd.get_dummies(raw, columns=one_hot_cat)

In [5]:
transformed_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,health,absences,M1,M2,...,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,activities_no,activities_yes,higher_no,higher_yes
0,18,4,4,2,2,0,3,6,25,30,...,0,0,0,0,0,1,1,0,0,1
1,17,1,1,1,2,0,3,4,25,25,...,0,0,0,1,0,0,1,0,0,1
2,15,1,1,1,2,3,3,10,35,40,...,0,0,0,1,0,0,1,0,0,1
3,15,4,2,1,3,0,5,2,75,70,...,0,0,0,0,1,0,0,1,0,1
4,16,3,3,1,2,0,5,4,30,50,...,0,0,0,1,0,0,1,0,0,1


In [18]:
transformed_data.columns.values

array(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
       'health', 'absences', 'M1', 'M2', 'M3', 'sex_F', 'sex_M',
       'address_R', 'address_U', 'Mjob_at_home', 'Mjob_health',
       'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'activities_no', 'activities_yes', 'higher_no', 'higher_yes'],
      dtype=object)

In [32]:
x = transformed_data.drop('M3', axis=1)

In [33]:
x.columns.values

array(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
       'health', 'absences', 'M1', 'M2', 'sex_F', 'sex_M', 'address_R',
       'address_U', 'Mjob_at_home', 'Mjob_health', 'Mjob_other',
       'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health',
       'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'activities_no',
       'activities_yes', 'higher_no', 'higher_yes'], dtype=object)

In [34]:
type(x)

pandas.core.frame.DataFrame

In [37]:
y = transformed_data['M3']
y.values

array([ 30,  30,  50,  75,  50,  75,  55,  30,  95,  75,  45,  60,  70,
        55,  80,  70,  70,  50,  25,  50,  75,  75,  80,  60,  40,  40,
        55,  75,  55,  55,  60,  85,  80,  60,  75,  30,  90,  75,  55,
        65,  55,  60,  90,  55,  45,  30,  55, 100,  70,  35,  65,  65,
        50,  55,  65,  50,  75,  75,  45,  80,  55,  55,  45,  45,  50,
        75,  60,  30,  40,  80,  75,  50,  25,  70,  55,  50,  50,  55,
        50,  25,  60,  55,  30,  75,  50,  40,  30,  70,  50,  35,  40,
        90,  30,  50,  70,  50,  75,  50,  70,  40,  25,  85,  70,  30,
        90,  55,  40,  90,  65,  80,  95,  50,  65,  95,  45,  80,  70,
        65,  40,  65,  75,  75,  65,  65,  40,  60,  55,  45,   0,  90,
         0,   0,  60,  55,   0,   0,   0,   0,  60,  75,   0,  45,  55,
        65,   0,  55,   0,  55,   0,  50,   0,  70,  50,   0,  60,  40,
        65,  50,  75,  60,   0,  35,   0,  50,  35,  60,  50,  80,   0,
        70,   0,  80,  50,   0,  45,  45,  55,  30,  45,  55,  4